In [0]:
% cd .. 
% cd root
import numpy as np
import pandas as pd
import tensorflow as tf
df = pd.read_csv("AL_clean_2.csv")
df.head()
df.fillna('', inplace=True)
print(len(df))
x = df["x1"].tolist()
y = df["y1"].tolist()

In [0]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Vectorize the data.
input_texts = []
target_texts = []
all_words = set()

for line in x:
    input_texts.append(['\t']+ str(line).split(" ")+ ['\n'])
    for word in str(line).split(" "):
        if word not in all_words:
            all_words.add(word)
for line in y:
  target_texts.append(['\t'] + str(line).split(" ") + ['\n'])
  for word in str(line).split(" "):
    if word not in all_words:
        all_words.add(word)
all_words.add("\t")
all_words.add("\n")

all_words = sorted(list(all_words))
num_tokens = len(all_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique tokens:', num_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

In [0]:
token_index = dict([(word, i) for i, word in enumerate(all_words)])
input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text):
        input_data[i, t] = token_index[word]
    for t, word in enumerate(target_text):
        output_data[i, t] = token_index[word]

dataset = tf.data.Dataset.from_tensor_slices((input_data, output_data)).shuffle(len(input_data)).batch(64, drop_remainder=True)
print(input_data.shape)
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape
reverse_index = dict((i, word) for word, i in token_index.items())

In [18]:
from keras.layers import Input, Embedding, LSTM, Dense, RepeatVector, Bidirectional, Dropout, Concatenate
from keras.optimizers import Adam, SGD
from keras.models import Model
import keras.backend as K

def negative_crossentropy(y_true, y_pred):
    return ( - K.categorical_crossentropy(y_true, y_pred))
def GANmodel(maxlen_input, latent_dim, word_embedding_size=100,dictionary_size=num_tokens):
  input_context = Input(shape=(maxlen_input,), dtype='int32', name='input_context')
  input_answer = Input(shape=(maxlen_input,), dtype='int32', name='input_answer')
  LSTM_encoder_bot = LSTM(latent_dim, init= 'lecun_uniform', name = 'encoder_bot')
  LSTM_decoder_bot = LSTM(latent_dim, init= 'lecun_uniform', name = 'decoder_bot')
  LSTM_encoder_discriminator = LSTM(latent_dim, init= 'lecun_uniform', trainable=False, name = 'encoder_discriminator')
  LSTM_decoder_discriminator = LSTM(latent_dim, init= 'lecun_uniform', trainable=False, name = 'decoder_discriminator')
  Shared_Embedding = Embedding(output_dim=word_embedding_size, input_dim=dictionary_size, input_length=maxlen_input, trainable=True, name = 'shared')

  word_embedding_context = Shared_Embedding(input_context)
  context_embedding_bot = LSTM_encoder_bot(word_embedding_context)
  word_embedding_answer = Shared_Embedding(input_answer)
  answer_embedding_bot = LSTM_decoder_bot(word_embedding_answer)
  context_embedding_discriminator = LSTM_encoder_discriminator(word_embedding_context)
  answer_embedding_discriminator = LSTM_decoder_discriminator(word_embedding_answer)

  merge_layer = Concatenate(axis=1)([context_embedding_bot, answer_embedding_bot])
  out = Dense(int(dictionary_size/2), activation="relu")(merge_layer)
  out = Dense(int(dictionary_size), activation="softmax")(out)
  loss = Concatenate(axis=1)([context_embedding_discriminator, answer_embedding_discriminator, out])
  loss = Dense(1, activation="sigmoid", trainable=False, name = 'discriminator_output')(loss)

  model = Model(input=[input_context, input_answer], output = [loss])
  return model
model = GANmodel(27, 128)
model.compile(loss=negative_crossentropy, optimizer="adam")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, name="encoder_bot", kernel_initializer="lecun_uniform")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, name="decoder_bot", kernel_initializer="lecun_uniform")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, trainable=False, name="encoder_discriminator", kernel_initializer="lecun_uniform")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, trainable=False, name="decoder_discriminator", kernel_initializer="lecun_uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Updat